# Assessment of number of measures

In [1]:
# required Python imports
import pandas as pd
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.3.19 imported


## Generate database

In [2]:
# generate Euler angles each 7 degrees and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179)))

In [3]:
dataset = pd.DataFrame(
    {
        "Euler1": database.euler[:, 0],
        "Euler2": database.euler[:, 1],
        "Euler3": database.euler[:, 2],
    }
)
dataset

,Euler1,Euler2,Euler3
0,0,0,0
1,0,0,7
2,0,0,14
3,0,0,21
4,0,0,28
...,...,...,...
4389,84,84,147
4390,84,84,154
4391,84,84,161
4392,84,84,168


In [4]:
# Transmission values for lambda 1987.29
Ta = 0.009882
Tb = 0.995815
Tc = 0.596951

# Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(Ta, Tb, Tc), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)


In [12]:
steps = (60., 36., 30., 25.73, 20.)  # in degrees
columns = ["6samples", "10samples", "12samples", "14samples", "18samples"]

for i, step in enumerate(steps):
    # create database
    angles = np.arange(0, 360, step)
    T_vals = []
    azi_vals = []

    for euler in database.euler:
        # rotate
        x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

        # extract XY intersection
        xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

        # get the indexes of specific angles
        indexes = f.find_nearest(xy_vectors["angles"], angles)

        # append values
        T_vals.append(xy_vectors.loc[indexes, ["T"]].T.values.tolist()[0])
        azi_vals.append(xy_vectors.loc[indexes, ["angles"]].T.values.tolist()[0])

    database.T_values = np.array(T_vals)
    database.azimuths = np.array(azi_vals)

    # estimate orientations
    disorientations = np.empty(database.euler.shape[0])

    for index, orientation in enumerate(database.euler):
        measures = np.column_stack(
            (
                database.T_values[index],
                database.azimuths[index],
                np.full_like(database.azimuths[index], 90),
            )
        )

        estimate = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc))
        disorientations[index] = f.calc_disorientation(orientation, estimate.x)

    
    # add column
    dataset[columns[i]] = disorientations

    # clean database
    del database.T_values
    del database.azimuths


In [13]:
dataset

,Euler1,Euler2,Euler3,6samples,10samples,12samples,14samples,18samples
0,0,0,0,0.000,0.799,0.000,0.000,0.000
1,0,0,7,0.000,0.000,0.000,0.000,0.000
2,0,0,14,0.000,0.000,0.000,0.000,0.244
3,0,0,21,0.000,0.000,0.000,0.237,0.258
4,0,0,28,0.001,0.000,0.000,0.227,0.258
...,...,...,...,...,...,...,...,...
4389,84,84,147,0.075,0.012,0.012,0.017,0.009
4390,84,84,154,0.011,0.010,0.009,0.012,0.009
4391,84,84,161,0.020,0.013,0.012,0.011,0.014
4392,84,84,168,0.015,0.011,0.011,0.009,0.011


## Generate dataframe and save data

In [14]:
from datetime import date    
today = date.today().isoformat()

In [15]:
dataset.to_csv(today + '_step7_lamb1987_measures.csv', index=False)

In [16]:
# quick statistical overview
dataset[columns].describe()

,6samples,10samples,12samples,14samples,18samples
count,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000
mean,0.098132,0.048677,0.035034,0.032710,0.026989
std,0.431543,0.196727,0.139178,0.174582,0.078444
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.004000,0.003000,0.003000,0.003000,0.003000
50%,0.009000,0.006000,0.005000,0.005000,0.004000
75%,0.028000,0.015000,0.013000,0.011000,0.011000
max,14.651000,5.957000,4.855000,8.890000,1.458000
